In [1]:
import numpy as np
import cvxpy as cp
from basis_generator import rand_moment

In [2]:
def single_behavior_visibility(X, X_basis):
    
    eta = cp.Variable((1, 1))
    alpha = cp.Variable((len(X_basis), 1))
    beta = cp.Variable((len(X_basis), 1))
    M = cp.Variable(X_basis[0].shape)
    N = cp.Variable(X_basis[0].shape)

    constraints = [N >> 0]
    constraints += [N == sum([beta[j]*X_basis[j] for j in range(len(X_basis))])]
    constraints += [N[0,0] == 1 - eta]

    for i in range(1,len(X)):
        constraints += [
            eta*X[i,i] + N[i,i] == M[i,i]
        ]

    constraints += [M >> 0]
    constraints += [M == sum([alpha[j]*X_basis[j] for j in range(len(X_basis))])]
    constraints += [M[0,0] == 1]

    prob = cp.Problem(cp.Maximize(eta),
                      constraints)
    prob.solve(solver=cp.MOSEK, verbose=False)
    
    coef = []
    for __ in range(3,len(constraints)-3):
        coef.append(np.real(constraints[__].dual_value[0][0]))
    
    return eta.value[0][0], coef

In [3]:
def NPA_bound(coef,X_basis):
    beta = cp.Variable((len(X_basis), 1))
    N = cp.Variable(X_basis[0].shape)

    constraints = [N >> 0]
    constraints += [N == sum([beta[j]*X_basis[j] for j in range(len(X_basis))])]
    constraints += [N[0,0] == 1]

    prob = cp.Problem(cp.Minimize(
            sum([N[j+1,j+1]*coef[j] for j in range(len(coef))])
            ),constraints)
    prob.solve(solver=cp.MOSEK, verbose=False)
    return prob.value

In [4]:
# Import basis
basis_filename = "data_basis/3-dim-2-num_obs-3-len_seq-3-num_out.npy"
X_basis = np.load(basis_filename)

In [5]:
dimX = 4
num_obs = 2
len_seq = 3
num_out = 3
remove_last_out = True

In [6]:
X, rho, P = rand_moment(dimX, num_obs, len_seq, num_out, [len_seq], remove_last_out)
eta, coef = single_behavior_visibility(X, X_basis)

In [ ]:
for __ in range(5):
    X_t, rho_t, P_t = rand_moment(dimX, num_obs, len_seq, num_out, [len_seq], remove_last_out)
    eta_t, coef_t = single_behavior_visibility(X_t, X_basis)
    if eta_t < eta:
        eta = eta_t
        coef = coef_t
        X = X_t
        rho = rho_t
        P = P_t

print(eta)

In [ ]:
np.real(np.diag(X)[1:] @ coef)

In [ ]:
NPA_bound(coef,X_basis)